# Simple template fitting using pyroofit

<div class="alert alert-block alert-danger">
    <b>Note for contributors:</b> Remember to run <code>Kernel > Restart & Clear output</code> before adding any changes to git!</div>

In this tutorial we will perform template fits with ``pyroofit``.

``pyroofit`` is a python wrapper for the ``RooFit`` package. Documentation can be found here: 
    http://www.desy.de/~swehle/pyroofit/
    
The project's code can be found at https://github.com/simonUU/PyrooFit

``pyroofit`` can be installed with ``pip3 install --user pyroofit``.

In [ ]:
import ROOT
from pyroofit.models import Gauss, Chebychev

import numpy as np
import pandas as pd

# Show pictures in this notebook
from IPython.display import Image

%matplotlib inline

In [ ]:
# Create some test data
data = pd.DataFrame(
    np.concatenate((
        np.random.normal(-2, 1, 1000), 
        np.random.normal(3, 2, 1000), 
        -5 + 10* np.random.random_sample(2000)
    )),
    columns=["x"]
)

<div class="alert alert-block alert-info">
    <b>Note:</b> We will fit the <b>histogram</b> of the above data, so note the conceptual difference to the <code>x, y</code> data from the tutorial <code>fitting_curves_data</code>.</div>

<div class="alert alert-block alert-success">
<b>Question [medium]:</b> Can you guess what the corresponding histogram to this data will look like and why?</div>

No? Then cheat and look at the histogram:

In [ ]:
data.hist(bins=30)

## First try: Fit single gaussian as signal and line as background

In [ ]:
# Gaussian for signal
pdf_sig = Gauss(('x', -5, 5), mean=(-1, 0, 1), title="Signal")

# Straight line for background
pdf_bkg = Chebychev(('x', -5, 5), n=1, title="Background")

<div class="alert alert-block alert-success">
<b>Question:</b> Why is the model for the straight line called 'Chebychev'?</div>

Now we build a compound PDF from the two simple PDFs.
``pyroofit`` is quite nice in that this has a very pythonic syntax (we overload the ``+`` operator):

In [ ]:
pdf = pdf_sig + pdf_bkg

Now we're ready to fit:

In [ ]:
pdf.fit(data)

Don't be deterred by the amount of output, but let's look at the results:

In [ ]:
pdf.get()

**Hint:** In order to get the results as a dictionary, use ``get_parameters()`` instead:

In [ ]:
pdf.get_parameters()

And plot:

In [ ]:
pdf.plot("test.png", legend=True)

In [ ]:
Image("test.png")

<div class="alert alert-block alert-success">
<b>Question 2 [easy]:</b> Why are the results so terrible?</div>

## Second try: Fit two gaussians as signal

In [ ]:
gauss1 = Gauss(('x', -5, 5), mean=(3, -5, 5), title="signal", name="gauss1")
gauss2 = Gauss(('x', -5, 5), mean=(-3, -5, 5), title="signal", name="gauss2")
pdf_sig = gauss1 + gauss2
pdf_bkg = Chebychev(('x', -5, 5), n=1, title="Background")
pdf = pdf_sig + pdf_bkg

In [ ]:
pdf.fit(data)

In [ ]:
print("Gauss 1:")
print(gauss1.get())
print("Gauss 2:")
print(gauss2.get())
print("Bkg:")
print(pdf_bkg.get())

In [ ]:
pdf.plot("test2.png", legend=True)

In [ ]:
Image("test2.png")

## Exercise 1

<div class="alert alert-block alert-success">
<b>Exercise 1 [easy]:</b> Fit one gaussian for signal and a linear background model to the following dataset:
</div>

In [ ]:
data = pd.DataFrame(
    np.concatenate((
        np.random.normal(-2, 1, 1000), 
        -5 + 10* np.random.random_sample(2000)
    )),
    columns=["x"]
)

## Fixing templates from MC

In the previous examples, we simply "knew" that our signal was shaped like a (two) Gaussian(s) and the background was linear.

Usually however, the situation isn't as simple and we first have to learn how our signal and background looks like by looking at MC data. Remember that in MC we always know signal from background (it's simulated data after all).

Thus, we can first fit our signal and background model to the MC, then fix the parameters. Now we have two 
PDFs $\mathrm{pdf}_\mathrm{sig}$ and $\mathrm{pdf}_\mathrm{bkg}$ and get the signal and background yields by
fitting the data with $\mu_\mathrm{sig}\mathrm{pdf}_\mathrm{sig} + \mu_\mathrm{bkg}\mathrm{pdf}_\mathrm{bkg}$.

In [ ]:
mc_signal = pd.DataFrame(
    np.random.normal(-2, 1, 1000),
    columns=["x"]
)

mc_bkg = pd.DataFrame(
    np.concatenate((
        np.random.normal(2, 1, 1000), 
        -5 + 10* np.random.random_sample(2000)
    )),
    columns=["x"]
)

data = pd.DataFrame(
    np.concatenate((
        np.random.normal(2, 1, int(1.2*1000)), 
        -5 + 10* np.random.random_sample(int(1.2*2000)),
        np.random.normal(-2, 1, int(0.3*1000))
    )),
    columns=["x"]
)

In [ ]:
mc_signal.hist(bins=30)

In [ ]:
mc_bkg.hist(bins=30)

In [ ]:
pdf_sig = Gauss(('x', -5, 5), mean=(3, -5, 5), title="signal", name="gauss1")

In [ ]:
pdf_sig.fit(mc_signal)

In [ ]:
pdf_sig.plot("mc_signal_fit.png", legend=True)

In [ ]:
Image("mc_signal_fit.png")

In [ ]:
pdf_sig.fix()

In [ ]:
pdf_bkg = Gauss(('x', -5, 5), mean=(3, -5, 5), title="Background", name="gauss2") + \
    Chebychev(('x', -5, 5), n=1, title="Background")

In [ ]:
pdf_bkg.fit(mc_bkg)

In [ ]:
pdf_bkg.plot("mc_bkg_fit.png", legend=True)

In [ ]:
Image("mc_bkg_fit.png")

In [ ]:
pdf_bkg.fix()

In [ ]:
pdf_bkg.get()

In [ ]:
pdf = pdf_sig + pdf_bkg

In [ ]:
pdf.fit(data)

In [ ]:
pdf.plot("fit_to_data.png", legend=True)

In [ ]:
Image("fit_to_data.png")

In [ ]:
pdf.get()

In [ ]:
0.3/(1.2+0.3)

In [ ]:
3.7e+02/(3.53e+03+3.7e+01)

In [ ]:
pdf_bkg.get()